### Se va a llevar a cabo el desarrollo del entrenamiento de 3 tipos de algoritmos basados en árboles
1. Random Forest
2. XGBoost
3. CatBoost
### Para ello, a demás de entrenarlo se va a presentar la matriz de confusión y las métricas obtenidas

## 1. Carga y Exploración del Dataset

- **Carga del CSV:**  
  Se leyó el archivo CSV para tener una primera visualización de los datos.

- **Exploración inicial:**  
  - Se mostraron las primeras filas del dataset.
  - Se revisó la información general (tipos de datos, cantidad de valores nulos) y se obtuvieron estadísticas descriptivas.

---

## 2. Análisis de las Variables

- **Identificación de columnas no informativas:**  
  - `id` y `LoanNr_ChkDgt` son identificadores y no aportan valor predictivo.
  - `Name` contiene información variable sin relevancia directa, salvo que se extraiga información adicional (como sector o tipo de empresa).

- **Variables de fecha:**  
  - `ApprovalDate` y `DisbursementDate` podrían generar fuga de información si se usan tal cual.  
  - Se recomienda transformarlas (por ejemplo, extrayendo el año, mes, etc.) o descartarlas.

---

## 3. Selección de Variables Relevantes

- **Variables potencialmente importantes para predecir la aprobación de un crédito:**  
  - **Características de la empresa:**  
    - `State`, `ApprovalFY`  
    - `NoEmp` (número de empleados)  
    - `NewExist` (empresa nueva o existente)  
    - `CreateJob` y `RetainedJob` (impacto en el empleo)  
    - `FranchiseCode`  
    - `UrbanRural`  
    - `RevLineCr`, `LowDoc`  
    - `DisbursementGross`, `BalanceGross`
  
  - **Variables relacionadas con el banco y la ubicación:**  
    - `Bank`, `BankState`, `City`  
    - Pueden ser relevantes pero es necesario codificarlas o agruparlas correctamente.

---

## 4. Análisis de Correlaciones

- **Matriz de correlación:**  
  Se calculó la correlación entre las variables numéricas y la variable objetivo `Accept` para identificar cuáles tienen mayor influencia en la aprobación del crédito.
  
- **Visualización:**  
  Se utilizó un mapa de calor para visualizar la matriz de correlación y facilitar la identificación de relaciones relevantes.

---

## 5. Preprocesamiento de Datos

- **Eliminación o transformación de columnas:**  
  - Se descartaron columnas como `id`, `LoanNr_ChkDgt` y `Name` en un primer análisis.
  - Las columnas de fechas se pueden transformar o eliminar para evitar data leakage.

- **Codificación de variables categóricas:**  
  - Se aplicó one-hot encoding a variables como `State`, `Bank`, `City` y `BankState` para convertirlas en variables numéricas.

- **Tratamiento de valores nulos:**  
  - Se identificaron y gestionaron los valores nulos, por ejemplo, mediante imputación simple (o descartándolos según el caso).

---

## 6. División del Dataset

- **Separación en conjuntos de entrenamiento y validación:**  
  - Se dividió el dataset en un conjunto de entrenamiento y uno de validación, asegurando mantener la proporción de la variable objetivo (`Accept`) para una evaluación representativa del modelo.

---


## 1. Random Forest

In [1]:
import pandas as pd

# Cargar el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')

# Mostrar las primeras filas
# print(df.head())
# Información general (tipos de datos y valores nulos)
#print(df.info())

# Estadísticas descriptivas
# print(df.describe(include='all'))
df

,id,LoanNr_ChkDgt,Name,City,State,Bank,BankState,ApprovalDate,ApprovalFY,NoEmp,...,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementDate,DisbursementGross,BalanceGross,Accept
0,bd9d6267ec5,1523195006,"P-SCAPE LAND DESIGN, LLC",NORTHFIELD,OH,CITIZENS BANK NATL ASSOC,RI,2005-11-01,2006,2,...,0,2,0,1,0.0,0.0,2005-12-31,8000.0,0.0,1
1,9eebf6d8098,1326365010,The Fresh & Healthy Catering C,CANTON,OH,"FIRSTMERIT BANK, N.A.",OH,2005-06-06,2005,2,...,1,2,1,1,0.0,0.0,2005-07-31,166000.0,0.0,1
2,83806858500,6179584001,AARON MASON & HOWE LLC,SAWYERWOOD,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,2003-03-18,2003,2,...,4,2,1,2,1.0,0.0,2003-03-31,25000.0,0.0,1
3,a21ab9cb3af,8463493009,MID OHIO CAR WASH,COLUMBUS,OH,THE HUNTINGTON NATIONAL BANK,OH,1995-06-28,1995,2,...,0,0,1,0,0.0,0.0,1996-01-31,220100.0,0.0,1
4,883b5e5385e,3382225007,Bake N Brew LLC,Newark,OH,THE HUNTINGTON NATIONAL BANK,OH,2009-04-16,2009,0,...,0,0,0,1,0.0,0.0,2009-05-31,25000.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22830,4f9443d2a46,1573725008,"SIBILA RACE ENGINEERING, INC",MASSILLON,OH,CITIZENS BANK NATL ASSOC,RI,2005-12-09,2006,1,...,0,1,0,1,0.0,0.0,2005-12-31,70000.0,0.0,1
22831,798db2753a7,2011184008,ENVIRO SHIELD POWER WASHING,SPRINGBORO,OH,"PNC BANK, NATIONAL ASSOCIATION",OH,1998-04-27,1998,2,...,0,0,1,0,0.0,1.0,1998-05-31,30000.0,0.0,1
22832,ddb3c5e9bff,4082983001,"MAINLINE TRCK&TRAILR SRVC, INC",BEDFORD,OH,GROWTH CAPITAL CORP.,OH,1990-05-09,1990,16,...,6,10,1,0,0.0,0.0,1991-02-13,92000.0,0.0,1
22833,407200a5dfe,7783283010,TIN BOX STUDIO,CINCINNATI,OH,KEYBANK NATIONAL ASSOCIATION,OH,1994-11-10,1995,1,...,0,0,1,0,0.0,1.0,1995-01-31,20000.0,0.0,1


In [17]:
approval_by_city = df.groupby('City')['Accept'].mean().sort_values(ascending=False) 
print("\nTasa de aprobación por ciudad:") 
print(approval_by_city.head(10))


Tasa de aprobación por ciudad:
City
AMSTERDAM                1.0
powell                   1.0
116 EAST THIRD STREET    1.0
ABERDEEN                 1.0
AMLIN                    1.0
Oak Harbor               1.0
Yorkshire                1.0
ZANESVIILLE              1.0
holland                  1.0
Woodmere                 1.0
Name: Accept, dtype: float64


Preprocesamos los datos

In [ ]:
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
# City, State, Name, Bank, BankState, ApprovalDate, , ApprovalDate, ApprovalFY, NoEmp, NewExist, UrbanRural,DisbursementDate, DisbursementGross

#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

print(df_clean.isnull().sum())
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)

# Mostrar la distribución original de la variable target 'Accept'
print("Distribución original de 'Accept':")
print(df_clean['Accept'].value_counts())

# Balancear el DataFrame:
# Supongamos que la clase 0 (rechazados) es la minoritaria y la clase 1 (aceptados) es la mayoritaria.

# Extraer los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]

# Realizar un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)

# Combinar ambas clases y mezclar los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)

# Verificar la nueva distribución
print("Distribución balanceada de 'Accept':")
print(df_balanced['Accept'].value_counts())

# df_balanced es el DataFrame final balanceado
df_clean = df_balanced

ApprovalFY       0
NoEmp            0
NewExist        14
CreateJob        0
RetainedJob      0
                ..
BankState_VA     0
BankState_WA     0
BankState_WI     0
BankState_WV     0
BankState_WY     0
Length: 2016, dtype: int64
Distribución original de 'Accept':
Accept
1    19004
0     3831
Name: count, dtype: int64
Distribución balanceada de 'Accept':
Accept
1    3831
0    3831
Name: count, dtype: int64


,ApprovalFY,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,DisbursementGross,...,BankState_SD,BankState_TN,BankState_TX,BankState_UT,BankState_Unknown,BankState_VA,BankState_WA,BankState_WI,BankState_WV,BankState_WY
0,2004,8,0.0,0,8,1,2,0.0,0.0,340000.0,...,False,False,False,False,False,False,False,False,False,False
1,2002,10,1.0,0,0,1,1,0.0,1.0,109200.0,...,False,False,False,False,False,False,False,False,False,False
2,1990,4,0.0,0,0,1,0,0.0,0.0,365000.0,...,False,False,False,False,False,False,False,False,False,False
3,2008,11,0.0,4,7,1,1,0.0,0.0,490000.0,...,False,False,False,False,False,False,False,False,False,False
4,2002,2,0.0,18,2,1,1,1.0,0.0,141472.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7657,2006,30,1.0,0,30,0,2,1.0,0.0,39275.0,...,False,False,False,False,False,False,False,False,False,False
7658,2003,7,0.0,0,0,1,1,0.0,0.0,406000.0,...,False,False,False,False,False,False,False,False,False,False
7659,2006,1,1.0,2,2,0,1,1.0,0.0,33400.0,...,False,False,False,False,False,False,False,False,False,False
7660,2011,40,0.0,10,40,0,1,1.0,0.0,100000.0,...,False,False,False,False,False,False,False,False,False,False


División del dataset

In [129]:
from sklearn.model_selection import train_test_split

X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


Entrenamiento Random Forest

In [130]:
from sklearn.ensemble import RandomForestClassifier

class_weights = {0: 15, 1: 1}  # Ajusta estos valores según tus datos
rf = RandomForestClassifier(n_estimators=100, random_state=42, class_weight=class_weights)

rf.fit(X_train, y_train)

RandomForestClassifier(class_weight={0: 15, 1: 1}, random_state=42)

Evaluación

In [131]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score

y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

print(f"Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")

# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

Matriz de Confusión:
[[567 200]
 [266 500]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       767
           1       0.71      0.65      0.68       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.70


## Y esto es para subirlo al kaggle

In [71]:
import pandas as pd
import datetime

# Cargar el dataset de test
df_test = pd.read_csv("../../../data/processed/df_test_nolabel.csv")

# --- Preprocesamiento ---
# Debes replicar exactamente los pasos que aplicaste al entrenamiento.
# 1. Eliminar las columnas irrelevantes (como hiciste en entrenamiento)
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_test_clean = df_test.drop(columns=cols_to_drop)

# 2. Aplicar one-hot encoding a las columnas categóricas que usaste en entrenamiento:
#    En tu entrenamiento usaste: ['Bank', 'City', 'BankState'] con drop_first=True.
df_test_clean = pd.get_dummies(df_test_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# 3. Imputar valores nulos (igual que en entrenamiento)
df_test_clean.fillna(0, inplace=True)

# --- Alinear las features ---
# Durante el entrenamiento, tu DataFrame final balanceado 'df_clean' tenía un cierto conjunto de columnas (features).
# Asumiendo que entrenaste el modelo con:
#   X_train = df_clean.drop('Accept', axis=1)
# Guarda la lista de features de entrenamiento:
features = list(df_clean.drop('Accept', axis=1).columns)
print("Número de features en entrenamiento:", len(features))

# Reindexa el DataFrame de test para que tenga exactamente las mismas columnas, llenando con 0 las que no estén.
df_test_clean = df_test_clean.reindex(columns=features, fill_value=0)
print("Número de features en test:", df_test_clean.shape[1])

# --- Predicción ---
# Extrae el array de features del test
X_test = df_test_clean.values

# Realiza las predicciones usando tu modelo entrenado (en este caso, 'rf')
df_test['Accept'] = voting_clf.predict(X_test)

# Asegúrate de que la columna 'Accept' sea de tipo entero
df_test['Accept'] = df_test['Accept'].astype(int)

# --- Exportar Submission ---
# Crea el archivo CSV con las columnas requeridas: 'id' y 'Accept'
filename = f"RandomFores_and_XGBoost_VotingClasiffier_UPDATE_colsample_bytree{datetime.datetime.now().strftime('%Y%m%d_%H_%M_%S')}.csv"
df_test.to_csv(filename, columns=['id', 'Accept'], index=False)

print("Archivo de submission 'my-model.csv' generado correctamente.")


Número de features en entrenamiento: 2015
Número de features en test: 2015


/home/angel/LBBYs/.venv/lib/python3.13/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


Archivo de submission 'my-model.csv' generado correctamente.


### Modificando los parámetros de configuración del modelo

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')

#################################Preprocesado##########################
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
# City, State, Name, Bank, BankState, ApprovalDate, , ApprovalDate, ApprovalFY, NoEmp, NewExist, UrbanRural,DisbursementDate, DisbursementGross
#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)
# Balancear el DataFrame:
# Extraer los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
# Realizar un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
# Combinar ambas clases y mezclar los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
# df_balanced es el DataFrame final balanceado
df_clean = df_balanced
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

#################################Entrenamiento del modelo##########################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

class_weights = {0: 1, 1: 1}  # Me emporan las métricas mucho 
rf = RandomForestClassifier(
    criterion = 'gini',        # entropy (bien) | log_loss (bien)  | gini (default)
    n_estimators=400,          # Más árboles para mayor estabilidad, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
    max_depth=80,              # Limita la profundidad para controlar el sobreajuste, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
    max_features='sqrt',       # log2 | sqrt (me da mejores resultados)
    class_weight=class_weights
    # bootstrap=True           # Usa muestreo con reemplazo

)
rf.fit(X_train, y_train)

###################################Métricas##########################################3
y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)
print(f"Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")
# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))
# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

Matriz de Confusión:
[[534 233]
 [229 537]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.70      0.70       767
           1       0.70      0.70      0.70       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.70


In [42]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')
#df = pd.read_csv('../../../data/processed/train_processed.csv')


#################################Preprocesado##########################
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
# City, State, Name, Bank, BankState, ApprovalDate, , ApprovalDate, ApprovalFY, NoEmp, NewExist, UrbanRural,DisbursementDate, DisbursementGross
#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)
# Balancear el DataFrame:
# Extraer los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
# Realizar un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
# Combinar ambas clases y mezclar los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
# df_balanced es el DataFrame final balanceado
df_clean = df_balanced
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

#################################Entrenamiento del modelo##########################
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

class_weights = {0: 5, 1: 2}  # Me emporan las métricas mucho 

rf = RandomForestClassifier(
    criterion = 'gini',        # entropy (bien) | log_loss (bien)  | gini (default)
    n_estimators=400,          # Más árboles para mayor estabilidad, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
    max_depth=80,              # Limita la profundidad para controlar el sobreajuste, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
    max_features='sqrt',       # log2 | sqrt (me da mejores resultados)
    class_weight=class_weights
    # bootstrap=True           # Usa muestreo con reemplazo

)
rf.fit(X_train, y_train)

###################################Métricas##########################################3
y_pred = rf.predict(X_test)

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)
print(f"Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")
# Reporte de Clasificación (precision, recall, f1-score, etc.)
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))
# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

Matriz de Confusión:
[[593 174]
 [284 482]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.77      0.72       767
           1       0.73      0.63      0.68       766

    accuracy                           0.70      1533
   macro avg       0.71      0.70      0.70      1533
weighted avg       0.71      0.70      0.70      1533

Macro F1-Score: 0.70


## Ahora pruebo el Grid para mejorarlo, no se mejora mucho 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')

# Eliminar columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Aplicar one-hot encoding a las columnas categóricas: Bank, City y BankState
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame (undersampling de la clase mayoritaria)
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. Dividir en conjuntos de entrenamiento y prueba (stratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Definir el grid de hiperparámetros para RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

# Crear un scorer basado en el Macro F1-Score para evaluar el balance entre las clases
scorer = make_scorer(f1_score, average='macro')

# 5. Configurar y ejecutar GridSearchCV
grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42, class_weight={0: 15, 1: 1}, n_jobs=10),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1
)
grid_rf.fit(X_train, y_train)

print("Mejores parámetros encontrados:")
print(grid_rf.best_params_)

# 6. Evaluar el mejor modelo en el conjunto de prueba
best_rf = grid_rf.best_estimator_
y_pred = best_rf.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Mejores parámetros encontrados:
{'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Matriz de Confusión:
[[567 200]
 [261 505]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.68      0.74      0.71       767
           1       0.72      0.66      0.69       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.6987897683319517


### Este es un random que le he metido la configuración de los hiperparámetros y además uso GridSearch

In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score, make_scorer
from sklearn.feature_selection import RFE

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame (undersampling de la clase mayoritaria)
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. Dividir en entrenamiento y prueba (stratificado)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Selección de características con RFE
# Usamos un RandomForestClassifier base para estimar la importancia de features
base_rf = RandomForestClassifier(
        criterion = 'gini',        # entropy (bien) | log_loss (bien)  | gini (default)
        n_estimators=400,          # Más árboles para mayor estabilidad, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
        max_depth=80,              # Limita la profundidad para controlar el sobreajuste, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
        max_features='sqrt'       # log2 | sqrt (me da mejores resultados)
        #class_weight=class_weights
    )
# Seleccionaremos, por ejemplo, las 2015 mejores features (no se cuenta la columna accept porque se droppeó)
rfe_selector = RFE(estimator=base_rf, n_features_to_select=2015, step=0.1)
rfe_selector.fit(X_train, y_train)

# Reducir X_train y X_test a las features seleccionadas
X_train_rfe = X_train.loc[:, rfe_selector.support_]
X_test_rfe = X_test.loc[:, rfe_selector.support_]

print("Número de features seleccionadas:", X_train_rfe.shape[1])

# 5. Definir el grid de hiperparámetros para el RandomForest con las features seleccionadas
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None]
}

scorer = make_scorer(f1_score, average='macro')

grid_rf = GridSearchCV(
    estimator=RandomForestClassifier(
        criterion = 'gini',        # entropy (bien) | log_loss (bien)  | gini (default)
        n_estimators=400,          # Más árboles para mayor estabilidad, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
        max_depth=80,              # Limita la profundidad para controlar el sobreajuste, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
        max_features='sqrt'       # log2 | sqrt (me da mejores resultados)
        #class_weight=class_weights
        ),
    param_grid=param_grid,
    scoring=scorer,
    cv=5,
    n_jobs=-1
)
grid_rf.fit(X_train_rfe, y_train)

print("Mejores parámetros encontrados:")
print(grid_rf.best_params_)

# 6. Evaluar el mejor modelo en el conjunto de prueba
best_rf = grid_rf.best_estimator_
y_pred = best_rf.predict(X_test_rfe)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Número de features seleccionadas: 2015
Mejores parámetros encontrados:
{'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 300}
Matriz de Confusión:
[[542 225]
 [240 526]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.69      0.71      0.70       767
           1       0.70      0.69      0.69       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.6966401442145038


## 2. XGBoost con los params tops del Random Forest

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

# 1. Cargar el dataset de entrenamiento
df = pd.read_csv('../../../data/processed/df_train.csv')


#############################Procesado##################################################
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)
# City, State, Name, Bank, BankState, ApprovalDate, ApprovalFY, NoEmp, NewExist, UrbanRural,DisbursementDate, DisbursementGross
#Codif variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)
# Ejemplo simple: imputar nulos con 0 (ajusta según cada variable)
df_clean.fillna(0, inplace=True)
# Balancear el DataFrame:
# Extraer los DataFrames de cada clase
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
# Realizar un muestreo aleatorio de la clase mayoritaria (1) para igualar el número de la minoritaria (0)
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
# Combinar ambas clases y mezclar los registros
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
# df_balanced es el DataFrame final balanceado
df_clean = df_balanced
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']
###############################################################################
# 5. Dividir el dataset de forma estratificada (80% entrenamiento, 20% prueba)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 6. Crear pesos de muestra para la clase deseada
# Se asigna un peso de 15 a la clase 0 (créditos rechazados) y 1 a la clase 1 (créditos aceptados)
sample_weights = y_train.map({0: 15, 1: 1})

# 7. Inicializar y entrenar el modelo XGBoost
xgb = XGBClassifier( 
                    random_state=42, 
                    n_estimators=400,          # Más árboles para mayor estabilidad, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
                    max_depth=80,              # Limita la profundidad para controlar el sobreajuste, NO PUEDO SUBIRLO MÁS PORQUE NO ALCANZA MÁS DE 0.7
                    max_features='sqrt',
                    use_label_encoder=False, 
                    eval_metric='logloss')
xgb.fit(X_train, y_train, sample_weight=sample_weights)

# 8. Realizar predicciones en el conjunto de prueba
y_pred = xgb.predict(X_test)

# 9. Evaluar el modelo

# Matriz de Confusión
cm = confusion_matrix(y_test, y_pred)
print("Matriz de Confusión:")
print(cm)

print("Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS")

# Reporte de Clasificación
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Calcular el Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")


/home/angel/LBBYs/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [17:23:15] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "max_features", "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Matriz de Confusión:
[[597 170]
 [308 458]]
Clase 0 SON CRÉDITOS RECHAZADOS y Clase 1 son CRÉDITOS ACEPTADOS
Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.66      0.78      0.71       767
           1       0.73      0.60      0.66       766

    accuracy                           0.69      1533
   macro avg       0.69      0.69      0.69      1533
weighted avg       0.69      0.69      0.69      1533

Macro F1-Score: 0.69


No me funciona catboost porque estoy usando la versión 3.13.1 y tiene que ser inferior a la versión 3.13

## 3. CatBoost

In [10]:
%pip install catboost

  Using cached catboost-1.2.7.tar.gz (71.5 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [143 lines of output]
        Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
        Using cached jupyterlab-3.6.8-py3-none-any.whl.metadata (12 kB)
        Using cached conan-1.66.0.tar.gz (789 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'done'
        Getting requirements to build wheel: started
        Getting requirements to build wheel: finished with status 'done'
        Preparing metadata (pyproject.toml): started
        Preparing metadata (pyproject.toml): finished with status 'done'
        Using cached ipython-9.0.2-py3-none-any.whl.metadata (4.3 kB)
        Using cached packaging-24.2-py3-none-any.

## Ensemble

### 1. Voting Ensemble 

Esta técnica combina las predicciones de ambos modelos. Puedes elegir entre:

* Hard Voting: Cada modelo vota por una clase y se toma la mayoría.

* Soft Voting: Se promedian las probabilidades de cada clase y se selecciona la de mayor probabilidad final.

Implementación con VotingClassifier:

In [70]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# 1. Cargar el dataset
df = pd.read_csv('../../../data/processed/df_train.csv')

############################# Procesado ################################
# Eliminar columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Convertir variables categóricas a dummy
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos (ajusta según tu necesidad)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame:
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target usando el DataFrame procesado
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. División de los datos
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Definir los dos modelos
rf = RandomForestClassifier(
    criterion='gini',
    n_estimators=500,
    max_depth=80,
    max_features='sqrt',
    class_weight={0:5, 1:2},
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=80,
    learning_rate=0.05,
    eval_metric='logloss',
    #gamma = 0.25,
    colsample_bytree = 0.6,
    random_state=42
)

# 5. Combinar modelos con VotingClassifier (voting='soft' para promediar probabilidades)
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb_model)],
    voting='soft',  # soft con peso 8 O 'hard' para voto mayoritario, con clase 0 con el peso 1
    n_jobs=-1
)

# 6. Entrenar el ensemble
voting_clf.fit(X_train, y_train)

# 7. Evaluar el ensemble
y_pred = voting_clf.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Matriz de Confusión:
[[578 189]
 [250 516]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.75      0.72       767
           1       0.73      0.67      0.70       766

    accuracy                           0.71      1533
   macro avg       0.71      0.71      0.71      1533
weighted avg       0.71      0.71      0.71      1533

Macro F1-Score: 0.7131642262423574


0.7018 para hard

0.7078 para el soft 0:8 y 1:2

0.70807 para el soft 0:5 y 1:2, joder así se consigue 0.71559 en el kaggle sus muertos
0.7131642262423574 modificando el colsample_bytree = 0.6 para conseguir en kaggle 0.71446


#### Le meto Gridsearch a ver si mejora #### 

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# 1. Cargar el dataset
df = pd.read_csv('../../../data/processed/df_train.csv')

############################# Procesado ################################
# Eliminar columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Convertir variables categóricas a dummy
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos (ajusta según tu necesidad)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. División de los datos (estratificada)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


# 4. Definir los modelos base
rf = RandomForestClassifier(
    criterion='gini',
    n_estimators=400,
    max_depth=80,
    max_features='sqrt',
    class_weight={0:1, 1:1},
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=80,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# 5. Combinar modelos con VotingClassifier
voting_clf = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb_model)],
    voting='hard',  # Empezamos con hard; se optimizará en GridSearchCV
    n_jobs=-1
)

# 6. Integrar GridSearchCV para optimizar el VotingClassifier
param_grid = {
    'voting': ['hard', 'soft'],
    'weights': [[1, 1], [8, 1], [15, 1]]
}

grid = GridSearchCV(voting_clf, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=1)
grid.fit(X_train, y_train)

print("Mejores parámetros:", grid.best_params_)
print("Mejor score (F1 macro):", grid.best_score_)

# 7. Evaluar el mejor modelo en el conjunto de prueba
best_model = grid.best_estimator_
y_pred = best_model.predict(X_test)

print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred))
print("Macro F1-Score:", f1_score(y_test, y_pred, average='macro'))


Fitting 5 folds for each of 6 candidates, totalling 30 fits


[19:15:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:15:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_la

Mejores parámetros: {'voting': 'hard', 'weights': [8, 1]}
Mejor score (F1 macro): 0.706076435685752
Matriz de Confusión:
[[540 227]
 [227 539]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       0.70      0.70      0.70       767
           1       0.70      0.70      0.70       766

    accuracy                           0.70      1533
   macro avg       0.70      0.70      0.70      1533
weighted avg       0.70      0.70      0.70      1533

Macro F1-Score: 0.7038485367356457


### 2. Stacking Ensemble

En el stacking, se entrena un meta-modelo (por ejemplo, una regresión logística) que recibe como entrada las predicciones (o probabilidades) de los modelos base.

In [54]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# 1. Cargar el dataset
df = pd.read_csv('../../../data/processed/df_train.csv')

############################# Procesado ################################
# Eliminar columnas irrelevantes
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Convertir variables categóricas a dummy
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar valores nulos (ajusta según tu necesidad)
df_clean.fillna(0, inplace=True)

# Balancear el DataFrame:
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced

# 2. Separar features y target usando el DataFrame procesado
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 3. División de los datos (con estratificación)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Definir los dos modelos base
rf = RandomForestClassifier(
    criterion='gini',
    n_estimators=400,
    max_depth=80,
    max_features='sqrt',
    class_weight={0:4, 1:1},
    random_state=42
)

xgb_model = xgb.XGBClassifier(
    n_estimators=400,
    max_depth=80,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

# 5. Definir el StackingClassifier con un meta-modelo (LogisticRegression)
estimators = [
    ('rf', rf),
    ('xgb', xgb_model)
]

stacking_clf = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),  # Meta-modelo; final_estimator: BaseEstimator (me creo un estimador propio) | LogisticRegression | None = None,
    cv=5,                                  # Número de folds para validación interna; cv: int | BaseCrossValidator | Iterable | str | None = None
    n_jobs=-1,
    stack_method= 'auto',                  # stack_method: Literal['auto', 'predict_proba', 'decision_function', 'predict'] = "auto", los 3 valores me dan
                                           # prácticamente lo mismo
    passthrough= False                     # passthrough: bool = False, en True me bajan todas las métricas
                                           # verbose: Int = 0
)

# 6. Entrenar el modelo stacking
stacking_clf.fit(X_train, y_train)

# 7. Evaluar el modelo stacking
y_pred_stack = stacking_clf.predict(X_test)

print("Matriz de Confusión (Stacking):")
print(confusion_matrix(y_test, y_pred_stack))
print("\nReporte de Clasificación (Stacking):")
print(classification_report(y_test, y_pred_stack))
print("Macro F1-Score (Stacking):", f1_score(y_test, y_pred_stack, average='macro'))


/home/angel/LBBYs/.venv/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [19:02:29] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
[19:02:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:02:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:02:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:02:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

[19:02:59] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



Matriz de Confusión (Stacking):
[[565 202]
 [250 516]]

Reporte de Clasificación (Stacking):
              precision    recall  f1-score   support

           0       0.69      0.74      0.71       767
           1       0.72      0.67      0.70       766

    accuracy                           0.71      1533
   macro avg       0.71      0.71      0.70      1533
weighted avg       0.71      0.71      0.70      1533

Macro F1-Score (Stacking): 0.7048517520215634


### Mejorando el que me da 0.70

In [2]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from imblearn.over_sampling import SMOTE
import pandas as pd

# 1. Cargar el dataset y preprocesar
df = pd.read_csv('../../../data/processed/df_train.csv')

#################################Preprocesado##########################
cols_to_drop = ['id', 'LoanNr_ChkDgt', 'Name', 'ApprovalDate', 'DisbursementDate', 'State']
df_clean = df.drop(columns=cols_to_drop)

# Codificación de variables categóricas
df_clean = pd.get_dummies(df_clean, columns=['Bank', 'City', 'BankState'], drop_first=True)

# Imputar nulos
df_clean.fillna(0, inplace=True)

# Balanceo de clases con SMOTE
df_accept_0 = df_clean[df_clean['Accept'] == 0]
df_accept_1 = df_clean[df_clean['Accept'] == 1]
n_minority = len(df_accept_0)
df_accept_1_under = df_accept_1.sample(n=n_minority, random_state=42)
df_balanced = pd.concat([df_accept_0, df_accept_1_under]).sample(frac=1, random_state=42).reset_index(drop=True)
df_clean = df_balanced
X = df_clean.drop('Accept', axis=1)
y = df_clean['Accept']

# 2. Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# 3. Balanceo de clases con SMOTE en entrenamiento
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# 4. Búsqueda de Hiperparámetros con GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 400, 800],
    'max_depth': [10, 30, 50, 80, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5],
    'max_features': ['sqrt', 'log2', None],
    'class_weight': [None, 'balanced']
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='f1_macro')
grid_search.fit(X_train_res, y_train_res)
print("Mejores parámetros:", grid_search.best_params_)

# 5. Modelo con mejores parámetros
best_rf = grid_search.best_estimator_

# 6. Entrenamiento con los mejores parámetros
best_rf.fit(X_train_res, y_train_res)

# 7. Predicciones
y_pred = best_rf.predict(X_test)

# 8. Métricas
print("Matriz de Confusión:")
print(confusion_matrix(y_test, y_pred))

print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred))

# Macro F1-Score
macro_f1 = f1_score(y_test, y_pred, average='macro')
print(f"Macro F1-Score: {macro_f1:.2f}")

# AUC-ROC
y_pred_proba = best_rf.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
print(f"AUC-ROC: {auc:.2f}")

# Validación cruzada
cv_scores = cross_val_score(best_rf, X, y, cv=5, scoring='f1_macro')
print(f"F1 Macro Cross-Validation: {cv_scores.mean():.2f} ± {cv_scores.std():.2f}")

# 9. Ajuste de umbral
threshold = 0.4  # Ajusta este umbral
y_pred_adjusted = (y_pred_proba >= threshold).astype(int)
print("Reporte de Clasificación con Umbral Ajustado:")
print(classification_report(y_test, y_pred_adjusted))


Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.3s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   2.2s
[CV] END class_weight=None, max_depth=10, max_features=sqrt, min_samples_leaf=1, min_samples_split=2,